In [1]:
from advectionGP import Model, EQ
import numpy as np

X = np.array([[0,2,3,4],[0,2,6,7],[0,1,3,7],[0,1,6,4],[2,4,3,4],[2,4,6,7],[2,3,3,7],[2,3,6,4]])
y = np.array([12,13,15,16,13,14,10,9])
boundary = ([0,0,0],[10,10,10])

k = EQ(1.0, 2.0)
m = Model(X,y,resolution=[2,2,2],boundary=boundary,N_feat=15,noise_std=5.0,kernel=k)

In [2]:
tloc = np.linspace(boundary[0][2],boundary[1][2],16)
xloc=np.linspace(0.5,7.5,5)
yloc=np.linspace(0.5,7.5,5)
sensN = len(xloc)*len(yloc)
obsN = len(tloc)
coords = np.asarray(np.meshgrid(xloc,yloc)).reshape(2,sensN)

In [3]:
coords.shape

(2, 25)